<a href="https://colab.research.google.com/github/pengbo7/rpg_e2vid/blob/master/TimeLens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TimeLens: Event-based Video Frame Interpolation

## Installation

First clone the repo and install an environment

In [1]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go
!nvidia-smi

Wed Dec 11 02:50:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Connect Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
print('Google Drive connected.')

Mounted at /content/gdrive
Google Drive connected.


In [3]:
!ls /content/gdrive

MyDrive


# Install Dependencies and Repo

In [4]:
%cd /content
!git clone https://github.com/uzh-rpg/rpg_timelens
!wget http://rpg.ifi.uzh.ch/timelens/data2/checkpoint.bin
!CUDA_VISIBLE_DEVICES=0

/content
Cloning into 'rpg_timelens'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 149 (delta 25), reused 14 (delta 14), pack-reused 110 (from 1)
Receiving objects: 100% (149/149), 73.59 MiB | 30.92 MiB/s, done.
Resolving deltas: 100% (60/60), done.
--2024-12-11 02:51:29--  http://rpg.ifi.uzh.ch/timelens/data2/checkpoint.bin
Resolving rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)... 130.60.183.133
Connecting to rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)|130.60.183.133|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://rpg.ifi.uzh.ch/timelens/data2/checkpoint.bin [following]
--2024-12-11 02:51:30--  https://rpg.ifi.uzh.ch/timelens/data2/checkpoint.bin
Connecting to rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)|130.60.183.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://download.ifi.uzh.ch/rpg/web/data/timelens/data2/check

In [5]:
################# Required Configurations ############################

#@markdown # Required Configuration
#@markdown Use the values in here to configure what you'd like TimeLens to do. If you do not have your own data, just leave the values below and it will download an example.

#@markdown ## Output file
#@markdown Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the filetype in the destination. `.mp4` is recommended for video input, `.gif` for gif inputs.
OUTPUT_FILE_PATH = "/content/rpg_timelens/out.mp4" #@param{type:"string"}

################# Optional configurations ############################

#@markdown # Optional Configuration
#@markdown Parameters below can be left with their defaults, but feel free to adapt them to your needs.

#@markdown ## Target FPS
#@markdown  how many frames per second should the result have. This will determine how many intermediate images are interpolated.
TARGET_FPS =  960#@param{type:"number"}

#@markdown ## Frame input directory
#@markdown A path, relative to your GDrive root, where you already have the list of frames in the format 00001.png, 00002.png, etc.
FRAME_INPUT_DIR = '/content/rpg_timelens/input_frames' #@param{type:"string"}
EVENTS_INPUT_DIR = '/content/rpg_timelens/input_events' #@param{type:"string"}

#@markdown ## Frame output directory
#@markdown A path, relative to your GDrive root, where you want the generated frame.
FRAME_OUTPUT_DIR = '/content/rpg_timelens/output_frames' #@param{type:"string"}


# Set Up Example
If you do not have data to process, just leave the default values in the form above and the cell below will download an example.

In [6]:
# Example
%%bash
wget http://rpg.ifi.uzh.ch/timelens/data2/example_github.zip -O /content/example_github.zip
unzip /content/example_github.zip -d /content/
rm -rf /content/example_github.zip
mv /content/example/events /content/rpg_timelens/input_events
mv /content/example/images /content/rpg_timelens/input_frames
mkdir /content/rpg_timelens/output_frames

Archive:  /content/example_github.zip
   creating: /content/example/
   creating: /content/example/events/
  inflating: /content/example/events/000358.npz  
  inflating: /content/example/events/000371.npz  
  inflating: /content/example/events/000368.npz  
  inflating: /content/example/events/000376.npz  
  inflating: /content/example/events/000383.npz  
  inflating: /content/example/events/000351.npz  
  inflating: /content/example/events/000353.npz  
  inflating: /content/example/events/000382.npz  
  inflating: /content/example/events/000365.npz  
  inflating: /content/example/events/000391.npz  
  inflating: /content/example/events/000354.npz  
  inflating: /content/example/events/000352.npz  
  inflating: /content/example/events/000356.npz  
  inflating: /content/example/events/000398.npz  
  inflating: /content/example/events/000389.npz  
  inflating: /content/example/events/000367.npz  
  inflating: /content/example/events/000381.npz  
  inflating: /content/example/events/000360

URL transformed to HTTPS due to an HSTS policy
--2024-12-11 02:53:19--  https://rpg.ifi.uzh.ch/timelens/data2/example_github.zip
Resolving rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)... 130.60.183.133
Connecting to rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)|130.60.183.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://download.ifi.uzh.ch/rpg/web/data/timelens/data2/example_github.zip [following]
--2024-12-11 02:53:20--  https://download.ifi.uzh.ch/rpg/web/data/timelens/data2/example_github.zip
Resolving download.ifi.uzh.ch (download.ifi.uzh.ch)... 130.60.61.200
Connecting to download.ifi.uzh.ch (download.ifi.uzh.ch)|130.60.61.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109862914 (105M) [application/zip]
Saving to: ‘/content/example_github.zip’

     0K .......... .......... .......... .......... ..........  0%  156K 11m29s
    50K .......... .......... .......... .......... ..........  0%  360K 8m13s
   100K .......... ....

In [9]:
# find the fps of the input images
import numpy as np
from os.path import join

timestamps = np.genfromtxt(join(FRAME_INPUT_DIR, "timestamp.txt"))
fps = int(1000000.0 / np.diff(timestamps).mean())
factor = int(TARGET_FPS / fps)
inserts = factor-1
print("Upsampling from ", fps, " to ", TARGET_FPS, " with ", inserts , " inserts.")



Upsampling from  159  to  960  with  5  inserts.


# Run TimeLens

In [10]:
%cd /content/rpg_timelens/
!mkdir $FRAME_OUTPUT_DIR
!python -m timelens.run_timelens ../checkpoint.bin $EVENTS_INPUT_DIR $FRAME_INPUT_DIR $FRAME_OUTPUT_DIR 0 $inserts

/content/rpg_timelens
mkdir: cannot create directory ‘/content/rpg_timelens/output_frames’: File exists
/content/rpg_timelens/timelens/warp_network.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to t

In [11]:
!ffmpeg -i "$FRAME_OUTPUT_DIR"/%06d.png timelens.mp4
from google.colab import files
files.download("timelens.mp4")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>